## SageMaker Pipeline

![ML pipeline](img/sagemaker_pipeline.png)

## Dataset

[SageMaker Examples](https://github.com/aws/amazon-sagemaker-examples/tree/main) To get relevant solutions to common tasks

The dataset you use is the [UCI Machine Learning Abalone Dataset](https://archive.ics.uci.edu/ml/datasets/abalone) [1].  The aim for this task is to determine the age of an abalone snail from its physical measurements. At the core, this is a regression problem.

![Abalone Snail](img/abalone.png)

The dataset contains several features: length, diameter, height, whole_weight, gender and rings.

The number of rings turns out to be a good approximation for age (age is rings + 1.5). However, to obtain this number requires cutting the shell through the cone, staining the section, and counting the number of rings through a microscope, which is a time-consuming task. However, the other physical measurements are easier to determine. You use the dataset to build a predictive model of the variable rings through these other physical measurements.

# Setup

In [17]:
! pip install -U sagemaker

# ADD USER ID

In [18]:
import sagemaker
import boto3
from botocore.exceptions import ClientError
from sagemaker.workflow.pipeline_context import PipelineSession
from dotenv import load_dotenv
import os
import json

print(load_dotenv('../.env'))

if not os.environ['HF_TOKEN']:
    raise("Please load token")

USER_ID = os.environ["UID"]
print(f"Your UID is {USER_ID}")

s3_bucket = os.environ['S3_WORKSHOP_BUCKET']
sess = sagemaker.Session(default_bucket=s3_bucket, default_bucket_prefix=USER_ID)

train_instance = 'ml.g5.2xlarge'
process_instance = 'ml.t3.xlarge'

preprocess_job_name = f"{USER_ID}-abalone-preprocess"
train_job_name = f"{USER_ID}-abalone-train"
eval_job_name = f"{USER_ID}-abalone-eval"
model_name = f"{USER_ID}-abalone-model"
model_package_group_name = f"{USER_ID}AbaloneModelPackageGroupName"
pipeline_name = f"{USER_ID}-AbalonePipeline"

region = "ap-south-1"
mean_square_error_threshold = 6.0

# s3_bucket = sess.default_bucket()
def create_bucket(bucket_name, region="ap-south-1"):
    s3_client = boto3.client('s3', region_name=region)
    try:
        location = {'LocationConstraint': region}
        s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
    except ClientError as e:
        print(f"Bucket {bucket_name} got response {e.response['Error']['Code']}")

create_bucket(s3_bucket)

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    # role = iam.get_role(RoleName=role_name)['Role']['Arn']
    role = "arn:aws:iam::009676737623:role/llmops_workshop_sagemaker_exec_role"

base_uri = f"s3://{s3_bucket}/{USER_ID}/abalone"
model_path = f"s3://{s3_bucket}/{USER_ID}/AbaloneTrain"

pipeline_session = PipelineSession(default_bucket=s3_bucket, default_bucket_prefix=USER_ID)

print(f"{role =}")
print(f"{s3_bucket =}")
print(f"{region =}")

True
Your UID is 7l4srb9
Bucket sagemaker-dhs-2025-llmops-workshop got response BucketAlreadyOwnedByYou
role ='arn:aws:iam::009676737623:role/service-role/AmazonSageMaker-ExecutionRole-20250814T174659'
s3_bucket ='sagemaker-dhs-2025-llmops-workshop'
region ='ap-south-1'


# Sync Dataset

In [19]:
! mkdir -p data

In [20]:
local_path = "data/abalone-dataset.csv"

s3 = boto3.resource("s3")
s3.Bucket(f"sagemaker-example-files-prod-{region}").download_file(
    "datasets/tabular/uci_abalone/abalone.csv", local_path
)

input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-dhs-2025-llmops-workshop/7l4srb9/abalone/abalone-dataset.csv


# Global Parameters

In [21]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="ProcessingInstanceType", default_value=process_instance)
train_instance_type = ParameterString(name="TrainingInstanceType", default_value=train_instance)
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="PendingManualApproval")
input_data = ParameterString(name="InputData",default_value=input_data_uri)
mse_threshold = ParameterFloat(name="MseThreshold", default_value=mean_square_error_threshold)

# Processing Step for Feature Engineering

 `scikit-learn` to do the following:

* Fill in missing data.
* Scale and normalize all numerical fields.
* Split the data into training, validation, and test datasets.

![ML pipeline](img/sagemaker_pipeline.png)


# PREPROCESSING FILE at code/preprocessing.py

In [22]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=instance_type,
    instance_count=processing_instance_count,
    base_job_name=preprocess_job_name,
    role=role,
    sagemaker_session=pipeline_session,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [23]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="code/preprocessing.py",
)

step_process = ProcessingStep(name="AbaloneProcess", step_args=processor_args)

/opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


# Training Step

[ProcessingJob Parameters](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html)

![ML pipeline](img/sagemaker_pipeline.png)

In [24]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=instance_type
)
# image_uri = "720646828776.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3"

xgb_train = Estimator(
    base_job_name=train_job_name,
    image_uri=image_uri,
    instance_type=train_instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=pipeline_session,
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=10,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
)

train_args = xgb_train.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.


In [25]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="AbaloneTrain",
    step_args=train_args,
)

## Evaluation Step

* Load the model.
* Read the test data.
* Issue predictions against the test data.
* Build a classification report, including accuracy and ROC curve.
* Save the evaluation report to the evaluation directory.

![ML pipeline](img/sagemaker_pipeline.png)

# EVALUATION FILE at code/evaluation.py

In [26]:
from sagemaker.processing import ScriptProcessor

script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=instance_type,
    instance_count=1,
    base_job_name=eval_job_name,
    role=role,
    sagemaker_session=pipeline_session,
)

eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="code/evaluation.py",
)

In [27]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)
step_eval = ProcessingStep(
    name="AbaloneEval",
    step_args=eval_args,
    property_files=[evaluation_report],
)

# Model Step
- Register vs Model vs Model Package
- ![ML pipeline](img/sagemaker_pipeline.png)

In [28]:
from sagemaker.model import Model

model = Model(
    name=model_name,
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

In [29]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep


model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=list(set(["ml.t2.medium", "ml.m5.xlarge", "ml.g5.2xlarge"])),
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)
step_register = ModelStep(name="AbaloneRegisterModel", step_args=register_args)

# Fail Step

In [30]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join

step_fail = FailStep(
    name="AbaloneMSEFail",
    error_message=Join(on=" ", values=["Execution failed due to MSE >", mse_threshold]),
)

# Condition Step

![ML pipeline](img/sagemaker_pipeline.png)

In [31]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

# Sagemaker helper function to extract values from Json documents
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=mse_threshold,
)

step_cond = ConditionStep(
    name="AbaloneMSECond",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[step_fail],
)

# Create Pipeline

In [32]:
from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        train_instance_type,
        model_approval_status,
        input_data,
        mse_threshold,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
    sagemaker_session=sess
)

# Start Pipeline

In [33]:
domain_tag = {
    "Key": "sagemaker:domain-arn",
    "Value": os.environ["AWS_SM_DOMAIN_ARN"],
}
pipeline.upsert(role_arn=role, tags=[domain_tag])

{'PipelineArn': 'arn:aws:sagemaker:ap-south-1:009676737623:pipeline/7l4srb9-AbalonePipeline',
 'ResponseMetadata': {'RequestId': '848002e9-f99d-4bbe-a7ab-bb5ce9116e87',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '848002e9-f99d-4bbe-a7ab-bb5ce9116e87',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Fri, 22 Aug 2025 23:14:50 GMT'},
  'RetryAttempts': 0}}

Start the pipeline and accept all the default parameters.

In [34]:
execution = pipeline.start(
    parameters={"MseThreshold": 3.0}
)

In [35]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-south-1:009676737623:pipeline/7l4srb9-AbalonePipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-south-1:009676737623:pipeline/7l4srb9-AbalonePipeline/execution/gqc6trwctla7',
 'PipelineExecutionDisplayName': 'execution-1755904490818',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2025, 8, 22, 23, 14, 50, 762000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 8, 22, 23, 14, 50, 762000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-south-1:009676737623:user-profile/d-yyezfeiz1rys/admin',
  'UserProfileName': 'admin',
  'DomainId': 'd-yyezfeiz1rys',
  'IamIdentity': {'Arn': 'arn:aws:sts::009676737623:assumed-role/AmazonSageMaker-ExecutionRole-20250814T174659/SageMaker',
   'PrincipalId': 'AROAQEQGHJRLRQV3OVWFN:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-south-1:009676737623:user-profile/d-yyezfeiz1rys/admin',
  'UserProfileName': 'admin',
  

In [ ]:
try:
    execution.wait()
    execution.list_steps()
except Exception as error:
    print(error)